<a href="https://colab.research.google.com/github/A1DS19/reddit_clone_ml/blob/master/reddit_comment_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow-text
!pip install tf-models-official

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.9 MB 3.8 MB/s 
     |████████████████████████████████| 578.0 MB 16 kB/s 
     |████████████████████████████████| 5.9 MB 74.7 MB/s 
     |████████████████████████████████| 1.7 MB 52.4 MB/s 
     |████████████████████████████████| 438 kB 86.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.8.0
    Uninstalling tensorflow-estimator-2.8.0:
      Successfully uninstalled tensorflow-estimator-2.8.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.8.0
    Uninstalling tensorboard-2.8.0:
      Successfully uninstalled tensorboard-2.8.0
  Attempting uninstall: keras
    Found existing installation: keras 2.8.0
    Uninstalling keras-2.8.0:
      Successfully uninstalled keras-2.8.0
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninsta

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow_hub as hub
import tensorflow_text as text
from official.nlp import optimization
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-9757da7c-9947-575a-784a-dc6f4ffe770f)


In [4]:
# import dataset
%%bash
pip install kaggle
mkdir ~/.kaggle
cp kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json
kaggle datasets download cosmos98/twitter-and-reddit-sentimental-analysis-dataset -f Reddit_Data.csv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/



mkdir: cannot create directory ‘/root/.kaggle’: File exists
100%|██████████| 2.45M/2.45M [00:00<00:00, 155MB/s]


In [5]:
!unzip Reddit_Data.csv.zip

Archive:  Reddit_Data.csv.zip
  inflating: Reddit_Data.csv         


In [6]:
!pip install fast_ml

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 163 kB/s 


In [7]:
raw_dataset = pd.read_csv('./Reddit_Data.csv')
raw_dataset.head(5)

,clean_comment,category
0,family mormon have never tried explain them t...,1
1,buddhism has very much lot compatible with chr...,1
2,seriously don say thing first all they won get...,-1
3,what you have learned yours and only yours wha...,0
4,for your own benefit you may want read living ...,1


In [8]:
raw_dataset.groupby('category').describe()

clean_comment                  
                 count unique   top freq
category                                
-1                8277   8250  wtf     8
 0               13042  12778        115
 1               15830  15771  lol     6

In [9]:
raw_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37249 entries, 0 to 37248
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   clean_comment  37149 non-null  object
 1   category       37249 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 582.1+ KB


In [10]:
df = raw_dataset.sample(frac=1, random_state=42)
df.head()

,clean_comment,category
31771,wish shake hands with these people one day,0
30819,why are they all fat,0
5100,this just being overly dramatic the ramblings ...,1
7646,you man which evident because you came trashse...,1
13781,posted date differs based the country the post...,-1


In [11]:
df.isnull().sum()

clean_comment    100
category           0
dtype: int64

In [12]:
df.isna().sum()

clean_comment    100
category           0
dtype: int64

In [13]:
df['category'].value_counts()

 1    15830
 0    13142
-1     8277
Name: category, dtype: int64

In [14]:
df['clean_comment'].to_numpy()

array([' wish shake hands with these people one day ',
       'why are they all fat ',
       'this just being overly dramatic the ramblings the uae foreign minister few months ago about the yemen participation pakistan was just done pressure pakistan into completely aligning with the gulf uae does not and will not make political statements that aren backed the rest the khaleeji countries suggest that uae pursuing more india focused diplomacy political future not true true from purely business perspective which happens the only thing uae rulers care about terms india pakistan relations the whole gwadar port business has pissed off these city states and they are pressuring pakistan abandon since will hurt them economically the use such politics just mask the real issue they have with the port the vague and contradictory stands pakistan and turkey are absolute proof that arab security — from libya yemen — the responsibility none but arab countries that statement cute the gulf arab countr

In [15]:
from sklearn.model_selection import train_test_split

X = df['clean_comment']
y = tf.keras.utils.to_categorical(df['category'].values, num_classes=3)
x_train, x_test, y_train, y_test = train_test_split(df['clean_comment'], y, test_size=0.25)

x_train[0], y_train[0]

(' family mormon have never tried explain them they still stare puzzled from time time like some kind strange creature nonetheless they have come admire for the patience calmness equanimity acceptance and compassion have developed all the things buddhism teaches ',
 array([0., 1., 0.], dtype=float32))

In [16]:
x_train = np.array([np.array(val) for val in x_train])
x_test = np.array([np.array(val) for val in x_test])

In [17]:
preprocessor = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-preprocess/2")
encoder = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder-cmlm/multilingual-base/1")

In [18]:
def get_embeddings(sentences):
  '''return BERT-like embeddings of input text
  Args:
    - sentences: list of strings
  Output:
    - BERT-like embeddings: tf.Tensor of shape=(len(sentences), 768)
  '''
  preprocessed_text = preprocessor(sentences)
  return encoder(preprocessed_text)['pooled_output']

In [19]:
get_embeddings([
    "Que es esto?",
    "What is this?"]
)

<tf.Tensor: shape=(2, 768), dtype=float32, numpy=
array([[ 0.16398181, -0.15170978,  0.1331759 , ...,  0.10207114,
        -0.06050967,  0.11483955],
       [ 0.15216757, -0.2748176 ,  0.24942039, ...,  0.18803118,
        -0.21796344,  0.12223694]], dtype=float32)>

In [20]:
e = get_embeddings(["banana", "bananos", "bike", "pistol", "machine gun"])

In [21]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity([e[0]], [e[1]]) #very similar
cosine_similarity([e[0]], [e[4]]) #not very similar(sus)
cosine_similarity([e[3]], [e[4]]) #similar

array([[0.90510905]], dtype=float32)

In [22]:
from keras import backend as K

def balanced_recall(y_true, y_pred):
    """This function calculates the balanced recall metric
    recall = TP / (TP + FN)
    """
    recall_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true_class, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        recall_by_class = recall_by_class + recall
    return recall_by_class / y_pred.shape[1]

def balanced_precision(y_true, y_pred):
    """This function calculates the balanced precision metric
    precision = TP / (TP + FP)
    """
    precision_by_class = 0
    # iterate over each predicted class to get class-specific metric
    for i in range(y_pred.shape[1]):
        y_pred_class = y_pred[:, i]
        y_true_class = y_true[:, i]
        true_positives = K.sum(K.round(K.clip(y_true_class * y_pred_class, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred_class, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        precision_by_class = precision_by_class + precision
    # return average balanced metric for each class
    return precision_by_class / y_pred.shape[1]

def balanced_f1_score(y_true, y_pred):
    """This function calculates the F1 score metric"""
    precision = balanced_precision(y_true, y_pred)
    recall = balanced_recall(y_true, y_pred)
    return 2 * ((precision * recall) / (precision + recall + K.epsilon()))

In [23]:
def build_classifier_model():
  i = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
  x = preprocessor(i)
  x = encoder(x)
  x = tf.keras.layers.Dropout(0.2, name="dropout")(x['pooled_output'])
  x = tf.keras.layers.Dense(3, activation='softmax', name="output")(x)
  return tf.keras.Model(i, x)

In [24]:
model = build_classifier_model()
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_word_ids':   0           ['text[0][0]']                   
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128),                                                      
                                 'input_mask': (Non                                               
                                e, 128)}                                                      

In [28]:
n_epochs = 20

METRICS = [
      tf.keras.metrics.CategoricalAccuracy(name="accuracy")
]

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss="categorical_crossentropy",
              metrics=METRICS)

In [29]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

callbacks = [ModelCheckpoint('./Checkpoints', 
                             save_best_only=True, 
                             save_weights_only=True),
             EarlyStopping(patience=2,
                           monitor="val_loss",
                           restore_best_weights=True),
             TensorBoard(log_dir="./logs")]

In [30]:
model_fit = model.fit(x_train, 
                      y_train, 
                      epochs=n_epochs,
                      batch_size=256,
                      validation_data=(x_test,y_test),
                      callbacks=callbacks)

Epoch 1/20
110/110 [==============================] - 403s 4s/step - loss: 0.8307 - accuracy: 0.6241 - val_loss: 0.7376 - val_accuracy: 0.6632
Epoch 2/20
110/110 [==============================] - 395s 4s/step - loss: 0.7697 - accuracy: 0.6499 - val_loss: 0.7350 - val_accuracy: 0.6714
Epoch 3/20
110/110 [==============================] - 386s 4s/step - loss: 0.7918 - accuracy: 0.6436 - val_loss: 0.7383 - val_accuracy: 0.6597
Epoch 4/20
110/110 [==============================] - 386s 4s/step - loss: 0.7680 - accuracy: 0.6531 - val_loss: 0.7744 - val_accuracy: 0.6606


In [31]:
def predict_class(reviews):
  '''predict class of input text
  Args:
    - reviews (list of strings)
  Output:
    - class (list of int)
  '''
  return [np.argmax(pred) for pred in model.predict(reviews)]

predict_class(["this is stupid...", 
               "this is awesome",
               "odio esto..."])

1/1 [==============================] - 1s 743ms/step


[0, 1, 0]

In [34]:
model.save('sentiment_classifier_v1')
model.save('sentiment_classifier_v1.h5')

In [36]:
from tensorflow import keras

new_model = keras.models.load_model("./sentiment_classifier_v1")

In [39]:
[np.argmax(pred) for pred in new_model.predict([
               "this is stupid...", 
               "this is awesome",
               "odio esto...",
               "I really like this"])]

1/1 [==============================] - 0s 34ms/step


[0, 1, 0, 1]

In [40]:
!zip 'sentiment_classifier.zip' /content/sentiment_classifier_v1

  adding: content/sentiment_classifier_v1/ (stored 0%)
